In [2]:
import pandas as pd
import numpy as np

# --- 1. Comprensión Inicial de los Datos ---
print("--- 1. Carga y Exploración Inicial ---")

# Cargar el dataset
# El archivo CSV tiene una columna de índice sin nombre al principio, la usamos como índice.
try:
    df = pd.read_csv('./data/data_short.csv', index_col=0)
except FileNotFoundError:
    print("Error: 'data_short.csv' no encontrado. Asegúrate de que el archivo está en el mismo directorio.")
    exit()

print("Primeras 5 filas del dataset:")
print(df.head())
print("\nInformación general del DataFrame (tipos de datos, nulos):")
df.info()
print("\nEstadísticas descriptivas para columnas numéricas:")
print(df.describe())
print("\nEstadísticas descriptivas para todas las columnas (incluyendo categóricas):")
print(df.describe(include='all'))
print("\nConteo de valores únicos por columna:")
for col in df.columns:
    print(f"Columna '{col}': {df[col].nunique()} valores únicos")

# --- 2. Limpieza de Datos ---
print("\n--- 2. Limpieza de Datos ---")

# a. Manejo de valores faltantes (NaNs)
print("\nValores faltantes por columna ANTES del tratamiento:")
print(df.isnull().sum())

# Vemos que 'Arrival Delay in Minutes' tiene valores faltantes.
# Estrategia: Imputar con la mediana, ya que los retrasos pueden tener outliers y la mediana es más robusta.
# Primero, asegurémonos de que la columna es numérica. A veces, los NaNs pueden estar mezclados con strings vacíos.
# En este caso, df.info() ya muestra que es float64, lo cual está bien.

# El archivo proporcionado tiene una celda vacía en la fila 213, columna 'Arrival Delay in Minutes'.
# Pandas la lee como NaN automáticamente si es un campo numérico, o como un string vacío si el tipo es 'object'.
# Si fuera un string vacío y la columna fuera 'object', necesitaríamos esto:
# df['Arrival Delay in Minutes'] = pd.to_numeric(df['Arrival Delay in Minutes'].replace('', np.nan), errors='coerce')

# Imputación con la mediana
median_arrival_delay = df['Arrival Delay in Minutes'].median()
print(f"\nMediana de 'Arrival Delay in Minutes': {median_arrival_delay}")
df['Arrival Delay in Minutes'].fillna(median_arrival_delay, inplace=True)

print("\nValores faltantes por columna DESPUÉS de la imputación:")
print(df.isnull().sum()) # Debería ser 0 para 'Arrival Delay in Minutes'

# b. Corrección de tipos de datos
# 'Arrival Delay in Minutes' es float64 debido a los NaNs originales. Ahora que no hay NaNs, podemos convertirla a int.
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].astype(int)
print("\nTipos de datos DESPUÉS de la corrección:")
df.info()

# c. Eliminación de columnas irrelevantes
# La columna 'id' es un identificador único para cada cliente y no aporta información predictiva generalizable.
# Podría ser útil para rastrear clientes específicos, pero no para el modelo en sí.
if 'id' in df.columns:
    df_cleaned = df.drop('id', axis=1)
    print("\nColumna 'id' eliminada.")
else:
    df_cleaned = df.copy()
    print("\nColumna 'id' no encontrada para eliminar.")


# --- 3. Transformación de Datos (Feature Engineering & Encoding) ---
print("\n--- 3. Transformación de Datos ---")
df_transformed = df_cleaned.copy()

# a. Codificación de variables categóricas
print("\nValores únicos en columnas categóricas ANTES de la codificación:")
categorical_cols = df_transformed.select_dtypes(include='object').columns
for col in categorical_cols:
    print(f"Columna '{col}': {df_transformed[col].unique()}")

# Variable Target: 'satisfaction'
# Es binaria: 'neutral or dissatisfied' y 'satisfied'. Mapearemos a 0 y 1.
satisfaction_map = {'neutral or dissatisfied': 0, 'satisfied': 1}
df_transformed['satisfaction'] = df_transformed['satisfaction'].map(satisfaction_map)
print("\nTarget 'satisfaction' mapeado a numérico.")

# Gender: 'Male', 'Female' -> 0, 1
gender_map = {'Male': 0, 'Female': 1} # o {'Female': 0, 'Male': 1}, la consistencia es la clave
df_transformed['Gender'] = df_transformed['Gender'].map(gender_map)
print("Columna 'Gender' mapeada a numérico.")

# Customer Type: 'Loyal Customer', 'disloyal Customer' -> 1, 0
customer_type_map = {'Loyal Customer': 1, 'disloyal Customer': 0}
df_transformed['Customer Type'] = df_transformed['Customer Type'].map(customer_type_map)
print("Columna 'Customer Type' mapeada a numérico.")

# Type of Travel: 'Personal Travel', 'Business travel' -> 0, 1
travel_type_map = {'Personal Travel': 0, 'Business travel': 1}
df_transformed['Type of Travel'] = df_transformed['Type of Travel'].map(travel_type_map)
print("Columna 'Type of Travel' mapeada a numérico.")

# Class: 'Eco', 'Eco Plus', 'Business'. Es ordinal.
class_map = {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
df_transformed['Class'] = df_transformed['Class'].map(class_map)
print("Columna 'Class' mapeada a numérico (ordinal).")

print("\nPrimeras filas del DataFrame después de la codificación:")
print(df_transformed.head())
print("\nInformación general del DataFrame DESPUÉS de la codificación:")
df_transformed.info() # Todas las columnas deberían ser numéricas ahora

# b. Creación de nuevas características (Feature Engineering) - Opcional por ahora, pero pensemos en una
# Podríamos crear una característica 'Total Delay'
df_transformed['Total Delay in Minutes'] = df_transformed['Departure Delay in Minutes'] + df_transformed['Arrival Delay in Minutes']
print("\nNueva característica 'Total Delay in Minutes' creada.")

# c. Escalado/Normalización de características numéricas
# Esto es importante para algoritmos sensibles a la escala de las características (ej. SVM, Regresión Logística, Redes Neuronales).
# Los modelos basados en árboles (Random Forest, XGBoost) no son tan sensibles.
# Como práctica general, es bueno escalar las características numéricas.
# No escalaremos la variable target 'satisfaction'.
# Las columnas ya codificadas a 0/1 (como Gender, Customer Type, etc.) tampoco necesitan escalado si usamos MinMaxScaler (ya están en un rango).
# Si usamos StandardScaler, sí se verían afectadas. Vamos a usar StandardScaler para las que no son binarias ni el target.

from sklearn.preprocessing import StandardScaler

# Seleccionar columnas numéricas para escalar (excluyendo las binarias ya codificadas y el target)
# Las columnas que son ratings (0-5) ya están en una escala similar, pero escalarlas no daña.
# 'Age', 'Flight Distance', las columnas de ratings y los delays son buenos candidatos.

cols_to_scale = ['Age', 'Flight Distance', 'Inflight wifi service',
                 'Departure/Arrival time convenient', 'Ease of Online booking',
                 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
                 'Inflight entertainment', 'On-board service', 'Leg room service',
                 'Baggage handling', 'Checkin service', 'Inflight service',
                 'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
                 'Total Delay in Minutes']

# Asegurarnos de que solo escalamos columnas que existen
cols_to_scale = [col for col in cols_to_scale if col in df_transformed.columns]


scaler = StandardScaler()
df_transformed[cols_to_scale] = scaler.fit_transform(df_transformed[cols_to_scale])

print("\nPrimeras filas del DataFrame DESPUÉS del escalado:")
print(df_transformed.head())
print("\nEstadísticas descriptivas DESPUÉS del escalado (columnas escaladas deben tener media ~0 y std ~1):")
print(df_transformed[cols_to_scale].describe())


# --- Guardar el dataset preprocesado ---
# Es una buena práctica guardar el resultado para no repetir estos pasos.
try:
    df_transformed.to_csv('data_short_preprocessed.csv', index=True) # Guardamos con índice por si es útil luego
    print("\nDataset preprocesado guardado como 'data_short_preprocessed.csv'")
except Exception as e:
    print(f"\nError al guardar el archivo: {e}")

print("\n--- FIN DEL PREPROCESAMIENTO INICIAL ---")
print("El DataFrame 'df_transformed' está listo para los siguientes pasos (división y entrenamiento).")

--- 1. Carga y Exploración Inicial ---
Primeras 5 filas del dataset:
       id  Gender      Customer Type  Age   Type of Travel     Class  \
0   70172    Male     Loyal Customer   13  Personal Travel  Eco Plus   
1    5047    Male  disloyal Customer   25  Business travel  Business   
2  110028  Female     Loyal Customer   26  Business travel  Business   
3   24026  Female     Loyal Customer   25  Business travel  Business   
4  119299    Male     Loyal Customer   61  Business travel  Business   

   Flight Distance  Inflight wifi service  Departure/Arrival time convenient  \
0              460                      3                                  4   
1              235                      3                                  2   
2             1142                      2                                  2   
3              562                      2                                  5   
4              214                      3                                  3   

   Ease of Online

C:\Users\Omar\AppData\Local\Temp\ipykernel_34060\4259285920.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Arrival Delay in Minutes'].fillna(median_arrival_delay, inplace=True)



Primeras filas del DataFrame DESPUÉS del escalado:
   Gender  Customer Type       Age  Type of Travel  Class  Flight Distance  \
0       0              1 -1.720453               0      1        -0.746893   
1       0              0 -0.922037               1      2        -0.968780   
2       1              1 -0.855502               1      2        -0.074331   
3       1              1 -0.922037               1      2        -0.646305   
4       0              1  1.473210               1      2        -0.989489   

   Inflight wifi service  Departure/Arrival time convenient  \
0               0.202156                           0.600208   
1               0.202156                          -0.707435   
2              -0.549354                          -0.707435   
3              -0.549354                           1.254030   
4               0.202156                          -0.053613   

   Ease of Online booking  Gate location  ...  On-board service  \
0                0.171324      -1

In [3]:
import pandas as pd
import numpy as np
from time import time

# Modelos de Clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
# Podríamos añadir XGBoost o LightGBM si están instalados, suelen dar buenos resultados
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier

# Utilidades de Scikit-learn
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# --- 1. Cargar Datos Preprocesados ---
print("--- 1. Cargando datos preprocesados ---")
try:
    # Asumiendo que guardaste el índice en el CSV anterior
    df_processed = pd.read_csv('data_short_preprocessed.csv', index_col=0)
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Error: 'data_short_preprocessed.csv' no encontrado.")
    print("Asegúrate de que el archivo del paso anterior está en el mismo directorio.")
    exit()
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    exit()

# --- 2. Preparar Datos para Modelado ---
print("\n--- 2. Preparando datos para modelado ---")
try:
    X = df_processed.drop('satisfaction', axis=1)
    y = df_processed['satisfaction']
    print(f"Forma de X (características): {X.shape}")
    print(f"Forma de y (target): {y.shape}")
    print("\nDistribución de la variable objetivo (satisfaction):")
    print(y.value_counts(normalize=True)) # Chequear desbalance
except KeyError:
    print("Error: La columna 'satisfaction' no se encuentra en el DataFrame.")
    print("Verifica que 'data_short_preprocessed.csv' se generó correctamente.")
    exit()
except Exception as e:
    print(f"Error al separar X e y: {e}")
    exit()

# --- 3. Definir Modelos y Estrategia de Evaluación ---
print("\n--- 3. Definiendo modelos y estrategia de evaluación ---")

# Lista de modelos a evaluar (con hiperparámetros por defecto o básicos)
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "SVC (Linear)": SVC(kernel='linear', probability=True, random_state=42), # probability=True para AUC
    "SVC (RBF)": SVC(kernel='rbf', probability=True, random_state=42),      # kernel RBF (Gaussiano)
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Gaussian Naive Bayes": GaussianNB(),
    # Añade aquí si tienes XGBoost/LightGBM:
    # "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    # "LightGBM": LGBMClassifier(random_state=42),
}

# Estrategia de Validación Cruzada
n_folds = 5
# Usamos StratifiedKFold para mantener la proporción de clases en cada fold
cv_strategy = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Métricas a calcular
scoring_metrics = ['accuracy', 'f1_weighted', 'roc_auc']

# --- 4. Evaluar Modelos ---
print(f"\n--- 4. Evaluando {len(models)} modelos usando {n_folds}-Fold Cross-Validation ---")
results = {}

for model_name, model in models.items():
    start_time = time()
    print(f"Evaluando: {model_name}...")
    try:
        # cross_validate permite calcular múltiples métricas a la vez
        cv_results = cross_validate(model, X, y, cv=cv_strategy, scoring=scoring_metrics, n_jobs=-1) # n_jobs=-1 usa todos los cores

        results[model_name] = {
            'Fit Time (mean)': cv_results['fit_time'].mean(),
            'Score Time (mean)': cv_results['score_time'].mean(),
            'Accuracy (mean)': cv_results['test_accuracy'].mean(),
            'Accuracy (std)': cv_results['test_accuracy'].std(),
            'F1 Weighted (mean)': cv_results['test_f1_weighted'].mean(),
            'F1 Weighted (std)': cv_results['test_f1_weighted'].std(),
            'AUC (mean)': cv_results['test_roc_auc'].mean(),
            'AUC (std)': cv_results['test_roc_auc'].std(),
        }
        elapsed_time = time() - start_time
        print(f"  Completado en {elapsed_time:.2f} segundos.")
        print(f"    AUC: {results[model_name]['AUC (mean)']:.4f} +/- {results[model_name]['AUC (std)']:.4f}")

    except Exception as e:
        print(f"  Error evaluando {model_name}: {e}")
        results[model_name] = {metric: np.nan for metric in scoring_metrics + ['fit_time', 'score_time']} # Marcar como NaN si falla

# --- 5. Consolidar y Rankear Resultados ---
print("\n--- 5. Resultados Consolidados y Ranking ---")

results_df = pd.DataFrame(results).T # Transponer para tener modelos como filas

# Ordenar por la métrica principal (AUC en este caso, puedes cambiar a F1 si prefieres)
results_df_sorted = results_df.sort_values(by='AUC (mean)', ascending=False)

print("\nRanking de Modelos (ordenado por AUC medio descendente):")
# Seleccionar y formatear columnas para mostrar
display_cols = ['AUC (mean)', 'AUC (std)', 'F1 Weighted (mean)', 'F1 Weighted (std)', 'Accuracy (mean)', 'Accuracy (std)', 'Fit Time (mean)']
print(results_df_sorted[display_cols].round(4))

# --- 6. Conclusiones Preliminares ---
print("\n--- 6. Conclusiones Preliminares ---")
best_model_name = results_df_sorted.index[0]
best_auc = results_df_sorted.iloc[0]['AUC (mean)']
print(f"\nEl modelo con mejor rendimiento (basado en AUC medio) en la validación cruzada es: {best_model_name} (AUC = {best_auc:.4f})")
print("\nPasos siguientes recomendados:")
print(f"1. Realizar ajuste de hiperparámetros (Hyperparameter Tuning) para los modelos top (ej., {best_model_name}, y quizás el 2º y 3º).")
print("2. Evaluar los modelos afinados en un conjunto de prueba (test set) separado para obtener una estimación final del rendimiento.")
print("3. Considerar la interpretabilidad del modelo y el tiempo de entrenamiento/predicción si son factores importantes.")

--- 1. Cargando datos preprocesados ---
Datos cargados correctamente.

--- 2. Preparando datos para modelado ---
Forma de X (características): (1000, 23)
Forma de y (target): (1000,)

Distribución de la variable objetivo (satisfaction):
satisfaction
0    0.56
1    0.44
Name: proportion, dtype: float64

--- 3. Definiendo modelos y estrategia de evaluación ---

--- 4. Evaluando 9 modelos usando 5-Fold Cross-Validation ---
Evaluando: Logistic Regression...
  Completado en 1.20 segundos.
    AUC: 0.9271 +/- 0.0159
Evaluando: KNN (k=5)...
  Completado en 0.71 segundos.
    AUC: 0.9246 +/- 0.0236
Evaluando: SVC (Linear)...
  Completado en 0.72 segundos.
    AUC: 0.9260 +/- 0.0157
Evaluando: SVC (RBF)...
  Completado en 0.70 segundos.
    AUC: 0.9530 +/- 0.0135
Evaluando: Decision Tree...
  Completado en 0.67 segundos.
    AUC: 0.8903 +/- 0.0173
Evaluando: Random Forest...
  Completado en 0.79 segundos.
    AUC: 0.9685 +/- 0.0118
Evaluando: Gradient Boosting...
  Completado en 0.76 segundos.


In [5]:
import pandas as pd
import numpy as np
from time import time

# Modelos y utilidades
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score # Para la métrica de scoring

# --- 1. Cargar Datos Preprocesados ---
print("--- 1. Cargando datos preprocesados ---")
try:
    # Asumiendo que guardaste el índice en el CSV anterior
    df_processed = pd.read_csv('data_short_preprocessed.csv', index_col=0)
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Error: 'data_short_preprocessed.csv' no encontrado.")
    print("Asegúrate de que el archivo del paso anterior está en el mismo directorio.")
    exit()
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    exit()

# --- 2. Separar Características (X) y Variable Objetivo (y) ---
print("\n--- 2. Separando X e y ---")
try:
    X = df_processed.drop('satisfaction', axis=1)
    y = df_processed['satisfaction']
    print(f"Forma inicial de X: {X.shape}")
    print(f"Forma inicial de y: {y.shape}")
except KeyError:
    print("Error: La columna 'satisfaction' no se encuentra en el DataFrame.")
    print("Verifica que 'data_short_preprocessed.csv' se generó correctamente.")
    exit()
except Exception as e:
    print(f"Error al separar X e y: {e}")
    exit()

# --- 3. Dividir Datos en Entrenamiento y Prueba ---
print("\n--- 3. Dividiendo datos en conjuntos de Entrenamiento y Prueba ---")
# Usamos stratify=y para mantener la proporción de clases 0 y 1 en ambos conjuntos
# Es crucial para que la evaluación sea representativa, especialmente si hay desbalance.
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,      # 20% de los datos para el conjunto de prueba
    random_state=42,     # Semilla para reproducibilidad
    stratify=y           # Mantiene la proporción de clases
)
print(f"Tamaño Entrenamiento: X={X_train.shape}, y={y_train.shape}")
print(f"Tamaño Prueba:        X={X_test.shape}, y={y_test.shape}")
print("\nDistribución de 'satisfaction' en Entrenamiento:")
print(y_train.value_counts(normalize=True))
print("\nDistribución de 'satisfaction' en Prueba:")
print(y_test.value_counts(normalize=True))

# --- 4. Ajuste de Hiperparámetros (Gradient Boosting) ---
print("\n--- 4. Ajustando hiperparámetros para Gradient Boosting Classifier ---")

# Modelo base
gb_classifier = GradientBoostingClassifier(random_state=42)

# Definir la "parrilla" de parámetros a probar.
# Es un punto de partida, se puede refinar/ampliar según los resultados y el tiempo disponible.
param_grid_gb = {
    'n_estimators': [100, 150, 200],        # Número de árboles
    'learning_rate': [0.05, 0.1, 0.15],     # Tasa de aprendizaje
    'max_depth': [3, 4, 5],                 # Profundidad máxima de cada árbol
    'subsample': [0.9, 1.0],                # Fracción de muestras por árbol (si < 1.0, es estocástico)
    # Otros parámetros que podrías añadir a la búsqueda:
    # 'min_samples_split': [2, 5],
    # 'min_samples_leaf': [1, 3],
    # 'max_features': ['sqrt', 'log2', None]
}

# Estrategia de Validación Cruzada (la misma que antes para consistencia)
n_folds = 5
cv_strategy = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Configurar GridSearchCV
# Optimizaremos usando 'roc_auc'. n_jobs=-1 usa todos los cores disponibles.
# verbose=2 muestra más información durante la ejecución.
grid_search_gb = GridSearchCV(
    estimator=gb_classifier,
    param_grid=param_grid_gb,
    scoring='roc_auc',      # Métrica a optimizar
    cv=cv_strategy,         # Estrategia de validación cruzada
    n_jobs=-1,              # Usar todos los cores disponibles
    verbose=2
)

# Realizar la búsqueda en los datos de ENTRENAMIENTO
print(f"\nIniciando GridSearchCV (probando {np.prod([len(v) for v in param_grid_gb.values()])} combinaciones)...")
start_time = time()
grid_search_gb.fit(X_train, y_train) # ¡¡IMPORTANTE: Usar SOLO datos de entrenamiento!!
end_time = time()
print(f"\nGridSearchCV completado en {(end_time - start_time):.2f} segundos.")

# --- 5. Mostrar Resultados del Ajuste ---
print("\n--- 5. Resultados del Ajuste de Hiperparámetros ---")
print(f"Mejor puntuación AUC (media en CV sobre datos de entrenamiento): {grid_search_gb.best_score_:.4f}")
print("Mejores hiperparámetros encontrados:")
print(grid_search_gb.best_params_)

# Guardar el mejor estimador encontrado por GridSearchCV
best_gb_model = grid_search_gb.best_estimator_

# --- 6. Próximos Pasos ---
print("\n--- 6. Próximos Pasos ---")
print("1. El objeto 'best_gb_model' contiene el Gradient Boosting Classifier entrenado con los mejores parámetros encontrados.")
print("   (Nota: GridSearchCV re-entrena automáticamente el mejor modelo con todos los datos de entrenamiento usados en el fit).")
print("2. El siguiente paso es EVALUAR el rendimiento de 'best_gb_model' en el CONJUNTO DE PRUEBA (X_test, y_test) para obtener una estimación final y no sesgada del rendimiento en datos no vistos.")
print("   Ejemplo: y_pred_test = best_gb_model.predict(X_test); test_auc = roc_auc_score(y_test, best_gb_model.predict_proba(X_test)[:, 1])")

--- 1. Cargando datos preprocesados ---
Datos cargados correctamente.

--- 2. Separando X e y ---
Forma inicial de X: (1000, 23)
Forma inicial de y: (1000,)

--- 3. Dividiendo datos en conjuntos de Entrenamiento y Prueba ---
Tamaño Entrenamiento: X=(800, 23), y=(800,)
Tamaño Prueba:        X=(200, 23), y=(200,)

Distribución de 'satisfaction' en Entrenamiento:
satisfaction
0    0.56
1    0.44
Name: proportion, dtype: float64

Distribución de 'satisfaction' en Prueba:
satisfaction
0    0.56
1    0.44
Name: proportion, dtype: float64

--- 4. Ajustando hiperparámetros para Gradient Boosting Classifier ---

Iniciando GridSearchCV (probando 54 combinaciones)...
Fitting 5 folds for each of 54 candidates, totalling 270 fits

GridSearchCV completado en 4.81 segundos.

--- 5. Resultados del Ajuste de Hiperparámetros ---
Mejor puntuación AUC (media en CV sobre datos de entrenamiento): 0.9797
Mejores hiperparámetros encontrados:
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 150, 'subsamp

In [7]:
import pandas as pd
import numpy as np
from time import time
import joblib # Para guardar el modelo

# Modelos y utilidades
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score # Para la métrica de scoring

# --- 1. Cargar Datos Preprocesados ---
print("--- 1. Cargando datos preprocesados ---")
try:
    df_processed = pd.read_csv('data_short_preprocessed.csv', index_col=0)
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Error: 'data_short_preprocessed.csv' no encontrado.")
    print("Asegúrate de que el archivo del paso anterior está en el mismo directorio.")
    exit()
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    exit()

# --- 2. Separar Características (X) y Variable Objetivo (y) ---
print("\n--- 2. Separando X e y ---")
try:
    X = df_processed.drop('satisfaction', axis=1)
    y = df_processed['satisfaction']
    print(f"Forma inicial de X: {X.shape}")
    print(f"Forma inicial de y: {y.shape}")
except KeyError:
    print("Error: La columna 'satisfaction' no se encuentra en el DataFrame.")
    print("Verifica que 'data_short_preprocessed.csv' se generó correctamente.")
    exit()
except Exception as e:
    print(f"Error al separar X e y: {e}")
    exit()

# --- 3. Dividir Datos en Entrenamiento y Prueba ---
print("\n--- 3. Dividiendo datos en conjuntos de Entrenamiento y Prueba ---")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,      # 20% de los datos para el conjunto de prueba
    random_state=42,     # Semilla para reproducibilidad
    stratify=y           # Mantiene la proporción de clases
)
print(f"Tamaño Entrenamiento: X={X_train.shape}, y={y_train.shape}")
print(f"Tamaño Prueba:        X={X_test.shape}, y={y_test.shape}")

# --- 4. Ajuste de Hiperparámetros (Gradient Boosting) ---
print("\n--- 4. Ajustando hiperparámetros para Gradient Boosting Classifier ---")

gb_classifier = GradientBoostingClassifier(random_state=42)

# Usaremos los parámetros óptimos que encontraste previamente si queremos saltar el GridSearch,
# o podemos dejar el GridSearch para asegurar que se encuentren. Dejaré el GridSearch por completitud.
# Si quieres usar directamente los parámetros encontrados:
# best_params_gb = {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 150, 'subsample': 0.9}
# best_gb_model = GradientBoostingClassifier(**best_params_gb, random_state=42)
# best_gb_model.fit(X_train, y_train) # Entrenar directamente con los mejores parámetros en todo X_train

# *** Dejando el GridSearchCV para el ejemplo completo ***
param_grid_gb = {
    'n_estimators': [100, 150, 200],        # Número de árboles
    'learning_rate': [0.05, 0.1, 0.15],     # Tasa de aprendizaje
    'max_depth': [3, 4, 5],                 # Profundidad máxima de cada árbol
    'subsample': [0.9, 1.0],                # Fracción de muestras por árbol
}
n_folds = 5
cv_strategy = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

grid_search_gb = GridSearchCV(
    estimator=gb_classifier,
    param_grid=param_grid_gb,
    scoring='roc_auc',
    cv=cv_strategy,
    n_jobs=-1,
    verbose=1 # Reducido el verbose para brevedad
)

print(f"\nIniciando GridSearchCV...")
start_time = time()
grid_search_gb.fit(X_train, y_train)
end_time = time()
print(f"\nGridSearchCV completado en {(end_time - start_time):.2f} segundos.")

# --- 5. Mostrar Resultados del Ajuste y Obtener Mejor Modelo ---
print("\n--- 5. Resultados del Ajuste de Hiperparámetros ---")
print(f"Mejor puntuación AUC (media en CV sobre datos de entrenamiento): {grid_search_gb.best_score_:.4f}")
print("Mejores hiperparámetros encontrados:")
print(grid_search_gb.best_params_)

# El mejor estimador ya está entrenado en todo X_train por GridSearchCV
best_gb_model = grid_search_gb.best_estimator_
print("\n'best_gb_model' contiene el modelo optimizado y entrenado.")

# --- 6. Guardar el Modelo Entrenado ---
print("\n--- 6. Guardando el modelo entrenado en archivo .pkl ---")
model_filename = 'gradient_boosting_satisfaction_model.pkl'
try:
    joblib.dump(best_gb_model, model_filename)
    print(f"Modelo guardado exitosamente como '{model_filename}'")
    print("Este archivo ahora puede ser cargado por tu API Flask.")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")

# (Opcional pero recomendado: Evaluación final en el Test Set)
# print("\n--- 7. Evaluación final en el Conjunto de Prueba ---")
# from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score
#
# y_pred_test = best_gb_model.predict(X_test)
# y_proba_test = best_gb_model.predict_proba(X_test)[:, 1] # Probabilidad de la clase positiva (1)
#
# print("Métricas en el conjunto de prueba:")
# print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
# print(f"F1 Score (Weighted): {f1_score(y_test, y_pred_test, average='weighted'):.4f}")
# print(f"AUC: {roc_auc_score(y_test, y_proba_test):.4f}")
# print("\nReporte de Clasificación detallado:")
# print(classification_report(y_test, y_pred_test))

print("\n--- PROCESO COMPLETO (Incluyendo guardado del modelo) ---")

# --- 7. Calcular Overfitting (Definitivo) ---
print("\n--- 7. Calculando Overfitting Definitivo ---")
from sklearn.metrics import roc_auc_score

# Rendimiento en Entrenamiento
y_proba_train = best_gb_model.predict_proba(X_train)[:, 1]
train_auc = roc_auc_score(y_train, y_proba_train)
print(f"AUC en Conjunto de Entrenamiento: {train_auc:.4f}")

# Rendimiento en Prueba (como se sugirió antes)
y_proba_test = best_gb_model.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_proba_test)
print(f"AUC en Conjunto de Prueba:        {test_auc:.4f}")

# Calcular diferencias
overfitting_abs = train_auc - test_auc
if train_auc > 0: # Evitar división por cero si el AUC de entrenamiento fuera 0
    overfitting_rel = (overfitting_abs / train_auc) * 100
    print(f"\nDiferencia Absoluta (Train - Test): {overfitting_abs:.4f}")
    print(f"Diferencia Relativa (Overfitting): {overfitting_rel:.2f}%")

    if overfitting_abs < 0.05: # O podrías usar overfitting_rel < 5.0
        print("\n¡El modelo cumple el criterio de overfitting < 5% (basado en diferencia absoluta < 0.05)! ✅")
    else:
        print("\nAdvertencia: El modelo NO cumple el criterio de overfitting < 5% (basado en diferencia absoluta >= 0.05). ❌")
else:
    print("\nNo se puede calcular overfitting relativo (AUC entrenamiento es 0).")

--- 1. Cargando datos preprocesados ---
Datos cargados correctamente.

--- 2. Separando X e y ---
Forma inicial de X: (1000, 23)
Forma inicial de y: (1000,)

--- 3. Dividiendo datos en conjuntos de Entrenamiento y Prueba ---
Tamaño Entrenamiento: X=(800, 23), y=(800,)
Tamaño Prueba:        X=(200, 23), y=(200,)

--- 4. Ajustando hiperparámetros para Gradient Boosting Classifier ---

Iniciando GridSearchCV...
Fitting 5 folds for each of 54 candidates, totalling 270 fits

GridSearchCV completado en 4.69 segundos.

--- 5. Resultados del Ajuste de Hiperparámetros ---
Mejor puntuación AUC (media en CV sobre datos de entrenamiento): 0.9797
Mejores hiperparámetros encontrados:
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 150, 'subsample': 0.9}

'best_gb_model' contiene el modelo optimizado y entrenado.

--- 6. Guardando el modelo entrenado en archivo .pkl ---
Modelo guardado exitosamente como 'gradient_boosting_satisfaction_model.pkl'
Este archivo ahora puede ser cargado por tu API 